In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
#URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
#dataframe = pd.read_csv(URL)
cols=['age', 'sex', 'cp', 'trestbps', 'chol','fbs', 'restecg', 'thalach',
        'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
dataframe = pd.read_csv('Heart.csv', index_col=0)
dataframe.head()

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
1,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
2,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
3,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
4,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
5,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


In [3]:
dataframe.columns

Index(['Age', 'Sex', 'ChestPain', 'RestBP', 'Chol', 'Fbs', 'RestECG', 'MaxHR',
       'ExAng', 'Oldpeak', 'Slope', 'Ca', 'Thal', 'AHD'],
      dtype='object')

In [4]:
dataframe = dataframe.rename(columns=dict(zip(dataframe.columns, cols)))

In [5]:
cp={'typical': 1, 'asymptomatic':4, 'nonanginal':3, 'nontypical':2}
dataframe['cp'] = dataframe['cp'].apply(lambda x: cp[x])

In [6]:
train, test = train_test_split(dataframe, test_size = 0.2)
train, val = train_test_split(train, test_size = 0.2)
print(len(train), 'Train')
print(len(val), 'Validation')
print(len(test), 'Test')

193 Train
49 Validation
61 Test


In [7]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_sizse = len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [9]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 303 entries, 1 to 303
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        299 non-null    float64
 12  thal      301 non-null    object 
 13  target    303 non-null    object 
dtypes: float64(2), int64(10), object(2)
memory usage: 35.5+ KB


In [8]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size = batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size = batch_size)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [36]:
dict(dataframe)

{'age': 1      63
 2      67
 3      67
 4      37
 5      41
        ..
 299    45
 300    68
 301    57
 302    57
 303    38
 Name: age, Length: 303, dtype: int64,
 'sex': 1      1
 2      1
 3      1
 4      1
 5      0
       ..
 299    1
 300    1
 301    1
 302    0
 303    1
 Name: sex, Length: 303, dtype: int64,
 'cp': 1      1
 2      4
 3      4
 4      3
 5      2
       ..
 299    1
 300    4
 301    4
 302    2
 303    3
 Name: cp, Length: 303, dtype: int64,
 'trestbps': 1      145
 2      160
 3      120
 4      130
 5      130
       ... 
 299    110
 300    144
 301    130
 302    130
 303    138
 Name: trestbps, Length: 303, dtype: int64,
 'chol': 1      233
 2      286
 3      229
 4      250
 5      204
       ... 
 299    264
 300    193
 301    131
 302    236
 303    175
 Name: chol, Length: 303, dtype: int64,
 'fbs': 1      1
 2      0
 3      0
 4      0
 5      0
       ..
 299    0
 300    1
 301    0
 302    0
 303    0
 Name: fbs, Length: 303, dtype: int64,